In [2]:
import os
from utils import *
import optuna

In [3]:
data, usermap, itemmap, users = load_data2()
data_train, data_val=split_data2(data, 0.2)

In [4]:
slim_el_study = optuna.create_study(
    study_name="SLIMElastic",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-07 20:29:59,785] Using an existing study with name 'SLIMElastic' instead of creating a new one.


In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender

slim_el_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(data_train, verbose=False)
slim_el_recommender.fit(**slim_el_study.best_params)

In [5]:
rp3beta_study = optuna.create_study(
    study_name="P3Beta",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-07 20:30:03,185] Using an existing study with name 'P3Beta' instead of creating a new one.


In [ ]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

rp3beta_recommender = RP3betaRecommender(data_train, verbose=False)
rp3beta_recommender.fit(**rp3beta_study.best_params)

#p3beta [topK: 35, alpha: 0.3598110596257423, beta: 0.1558338653772352, implicit: True, normalize_similarity: True]

In [10]:
from Recommenders.BaseRecommender import BaseRecommender
import scipy.sparse as sps
from numpy import linalg as LA

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1/norm*alpha + R2/norm*(1-alpha) where R1 and R2 come from
    algorithms trained on different loss functions.

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
        
    def fit(self, norm, alpha = 0.5):

        self.alpha = alpha
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        
        
        if norm_item_weights_1 == 0:
            norm_item_weights_1 = 0.0000001
            #raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            norm_item_weights_2 = 0.0000001
            #raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * (1-self.alpha)

        return item_weights



In [6]:
rat_rank_study = optuna.create_study(
    study_name="rat_rank",
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-07 20:30:06,639] Using an existing study with name 'rat_rank' instead of creating a new one.


In [ ]:
def objective(trial):
    norm = trial.suggest_categorical("norm", [1, 2, np.inf, -np.inf])
    alpha = trial.suggest_float("alpha", 0, 1)

    recommender_object = DifferentLossScoresHybridRecommender(data_train, slim_el_recommender, rp3beta_recommender)
    recommender_object.fit(
        norm = norm, 
        alpha = alpha)

    _, _, ev_map, _, _ = evaluator(recommender_object, data_train, data_val)

    return ev_map

rat_rank_study.optimize(objective, n_trials=500)

In [7]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender

slim_el_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(data_train+data_val, verbose=False)
slim_el_recommender.fit(**slim_el_study.best_params)

In [8]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

rp3beta_recommender = RP3betaRecommender(data_train+data_val, verbose=False)
rp3beta_recommender.fit(**rp3beta_study.best_params)

In [11]:
recommender_object = DifferentLossScoresHybridRecommender(data_train+data_val, slim_el_recommender, rp3beta_recommender)
recommender_object.fit(**rat_rank_study.best_params)

In [ ]:
submission2(recommender_object, users, usermap, itemmap, data_train+data_val)